In [4]:
# import pandas as pd
# import plotly.express as px
# import json

# # Load the data
# with open('mock_activities.json', 'r') as f:
#     data = json.load(f)

# df = pd.DataFrame(data)
# df


,id,googlePlaceId,name,description,lat,lng,address,city,category,averageDuration,rating,opening_hours
0,1,ChIJLU7jZClu5kcR8OjaNoJ1Ft4,Eiffel Tower,Iconic iron lattice tower on the Champ de Mars.,48.8584,2.2945,"Champ de Mars, 5 Av. Anatole France",Paris,landmark,120,4.7,"{'0': {'open': '09:30', 'close': '22:45'}, '1'..."
1,2,ChIJD7fiBh9u5kcRY9kr_GHbg4Y,Louvre Museum,The world's largest art museum and a historic ...,48.8606,2.3376,Rue de Rivoli,Paris,museum,180,4.8,"{'0': {'open': '09:00', 'close': '18:00'}, '1'..."
2,3,ChIJ90_Y9RRu5kcRIn_Xv6pT_98,Musée d'Orsay,Museum in a grand railway station holding Fren...,48.8599,2.3265,1 Rue de la Légion d'Honneur,Paris,museum,150,4.7,"{'0': {'open': '09:30', 'close': '18:00'}, '1'..."
3,4,ChIJ4z_pq_Vv5kcRN_uT8S5fW6w,Cathédrale Notre-Dame de Paris,Medieval Catholic cathedral on the Île de la C...,48.8530,2.3499,6 Parvis Notre-Dame - Pl. Jean-Paul II,Paris,landmark,60,4.7,"{'0': {'open': '08:00', 'close': '18:45'}, '1'..."
4,5,ChIJ9U9_9RRu5kcRIn_Xv6pT_99,Sainte-Chapelle,13th-century Gothic chapel with stunning stain...,48.8554,2.3450,10 Bd du Palais,Paris,landmark,45,4.8,"{'0': {'open': '09:00', 'close': '17:00'}, '1'..."
...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,ChIJT_NNN_NNNBkcRIn_Xv6pT_LA36,Guisados,Popular stop for homestyle braised-meat tacos ...,34.0700,-118.2500,1261 Sunset Blvd,Los Angeles,restaurant,45,4.6,"{'0': {'open': '09:00', 'close': '20:00'}, '1'..."
96,97,ChIJT_OOO_OOOBkcRIn_Xv6pT_LA37,Sushi Kaneyoshi,High-end Michelin-starred sushi omakase.,34.0480,-118.2380,111 S San Pedro St,Los Angeles,restaurant,150,4.9,"{'0': None, '1': None, '2': {'open': '19:00', ..."
97,98,ChIJT_PPP_PPPBkcRIn_Xv6pT_LA38,République,French bistro and bakery in a historic gothic ...,34.0619,-118.3440,624 S La Brea Ave,Los Angeles,restaurant,90,4.7,"{'0': {'open': '08:00', 'close': '14:00'}, '1'..."
98,99,ChIJT_QQQ_QQQBkcRIn_Xv6pT_LA39,Grand Hope Park,Peaceful park in the South Park district of Do...,34.0450,-118.2600,919 S Grand Ave,Los Angeles,park,45,4.5,"{'0': {'open': '00:00', 'close': '23:59'}, '1'..."


In [14]:
# # Create the Plotly Map
# config = {'scrollZoom': True}
# fig = px.scatter_mapbox(
#     df, 
#     lat="lat", 
#     lon="lng", 
#     hover_name="name", 
#     hover_data=["category", "rating", "averageDuration"],
#     color="category",        # Color points by category (museum, park, etc.)
#     zoom=10, 
#     height=800,
#     mapbox_style='carto-positron'
# )

# # 4. Update layout to use OpenStreetMap (No API key needed)
# fig.update_layout(mapbox_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# fig.show()

In [1]:
from dotenv import load_dotenv
import os

load_dotenv("../.env.local")  # explicitly load the file
print("API KEY FOUND:", os.getenv("GEMINI_API_KEY") is not None)

API KEY FOUND: True


In [2]:
import datetime

def log_itinerary(user_input, response, time_taken):
    # 1. Extract the parsed dictionary
    itinerary_data = response.parsed
    
    # 2. Format it as a pretty JSON string
    pretty_json = json.dumps(itinerary_data, indent=4)
    
    # 3. Create the log entry
    log_entry = f"""
        {'='*60}
        USER INPUT: {user_input}
        TIME TAKEN: {time_taken}
        TIMESTAMP:  {datetime.datetime.now()}
        {'-'*60}
        GENERATED ITINERARY:
        {pretty_json}
        {'='*60}
"""
    
    # Save to a file to keep a history
    with open("itinerary_history.log", "a", encoding="utf-8") as f:
        f.write(log_entry)

In [22]:
import json
from typing import List, Optional
from pydantic import BaseModel, Field
from google import genai
from google.genai import types
import time

# --- SCHEMAS (Matches Drizzle structure) ---
class ItineraryItem(BaseModel):
    title: str
    google_place_id: str
    start_time: str = Field(description="ISO format or HH:mm")
    end_time: str
    type: str = Field(description="e.g., 'activity', 'restaurant', 'commute'")
    commute_info: Optional[str] = Field(None, description="e.g. '15 min drive'")
    commute_seconds: Optional[int] = 0
    next_item_id: Optional[str] = Field(None, description="ID of the next item in the chain")

class DayPlan(BaseModel):
    day_number: int
    items: List[ItineraryItem]

class FullItinerary(BaseModel):
    days: List[DayPlan]
        
class ItineraryGenerator:
    def __init__(self):
        
        self.client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
        self.model_id = "gemini-3-flash-preview"

    def generate(self, user_picks: List[dict], num_days: int, current_itinerary: List = []):
        
        prompt = f"""
        Role: Expert Travel Planner
        
        Input:
        - User Picks: {json.dumps(user_picks)}
        - Number of Days: {num_days}
        - Current State: {json.dumps(current_itinerary)}
        
        Task:
        1. Distribute the items from User Picks across {num_days} days based on geographic proximity.
        2. Make the full-day itinerary
        3. For each Day, sequence the activities logically (e.g., Breakfast -> Activity -> Lunch -> Activity -> Dinner).
        """
        
        contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]

        start_time = time.time()
        response = self.client.models.generate_content(
            model=self.model_id,
            contents=contents,
            config={
                "response_mime_type": "application/json",
                "response_json_schema": FullItinerary.model_json_schema(),
            },
        )
        time_taken = time.time()-start_time
        
        return response, time_taken

if __name__ == "__main__":
    # Mock user data
    picks = [
    {"name": "Hollywood Walk of Fame", "lat": 34.1016, "lng": -118.3267, "place_id":"ChIJXyC7WTu_woARPvVMCHBXd4U"},
    {"name": "Angel's Flight Railway", "lat": 34.0513, "lng": -118.2493, "place_id": "ChIJ_SlqhEzGwoARrIIERV62_po"},
    {"name": "The Last Bookstore", "lat": 34.0477, "lng": -118.2496, "place_id": "ChIJ7YJ_IkrGwoARNak4CDihkYU"},
    {"name": "The Broad", "lat": 34.0544, "lng": -118.2506, "place_id": "ChIJXaYsEk3GwoARvx7RKBUE8Zg"},
    {"name": "Santa Monica Pier", "lat": 34.0099, "lng": -118.4960, "place_id": "ChIJm6deTdekwoARTY_RzhoRms0"},
    {"name": "Griffith Observatory", "lat": 34.1184, "lng": -118.3004, "place_id": "ChIJywjU6WG_woAR3NrWwrEH_3M"},
    {"name": "Venice Beach Boardwalk", "lat": 33.9866, "lng": -118.4740, "place_id": "ChIJsyUr_9y7woARG6pTv7ogvu0"},
    {"name": "Runyon Canyon Park", "lat": 34.1054, "lng": -118.3486, "place_id": "ChIJhX1uVuC-woARQNzq4-b1Prk"},
    {"name": "The Huntington Library", "lat": 34.1292, "lng": -118.1145, "place_id": "ChIJc1xhAk3bwoARBBlfKQzXKLM"},
    {"name": "Joshua Tree", "lat": 33.8734, "lng": -115.9010, "place_id": "ChIJe6hluYWP2oAR4p3rOqftdxk"},]

#     picks = [{"name": "Hollywood Walk of Fame", "lat": 34.1016, "lng": -118.3267},
#             ]
    generator = ItineraryGenerator()
    
    print("Generating Itinerary...")
    result_json, time_taken = generator.generate(picks, num_days=2)
    
    print("\n--- FINAL GENERATED ITINERARY ---")
    print(f"\nTime spent: {time_taken}")
    print(result_json)
    
    print("\n--- LOG GENERATED OUTPUT ---")
    log_itinerary(picks, result_json, time_taken)

Generating Itinerary...

--- FINAL GENERATED ITINERARY ---
sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text="""{
  "days": [
    {
      "day_number": 1,
      "items": [
        {
          "title": "Musso & Frank Grill",
          "google_place_id": "ChIJN09N-vS-woAR_y4-rWjN-Y8",
          "start_time": "09:00",
          "end_time": "10:30",
          "type": "restaurant",
          "commute_info": "5 min walk",
          "commute_seconds": 300,
          "next_item_id": "item_h_walk"
        },
        {
          "title": "Hollywood Walk of Fame",
          "google_place_id": "ChIJL-m_q_6-woAR_z6uD6n_p-E",
          "start_time": "10:30",
          "end_time": "12:30",
          "type": "activity",
          "commute_info": "5 min walk",
          "commute_seconds": 300,
          "next_item_id": "item_stout"
        },
        {
          "title": "Stout Burgers & Beers",
          "google_pla